# transformersのライブラリを利用したtfのBERT事前学習モデルのfinetuneのサンプル

ポイント

- transformers, datasetsなどのライブラリに頼ることで、コードが短くわかりやすい
- transformersはtensorflowのモデルをpytorchに持ち込める
- livedoor ニュースコーパスの文章分類をdownstream taskとした

## ライブラリ
pythonの仮想環境、パッケージはpoetryを使用しました  
pyproject.tomlを見てもらえるとわかりますが、特にこのnotebookで使用したのは以下になるとおもいます。

```
python = "^3.7.1"
jupyterlab = "^3.1.6"
ipywidgets = "^7.6.3"
torch = "^1.9.0"
transformers = "^4.9.2"
sentencepiece = "^0.1.96"
datasets = "^1.11.0"
```

In [23]:
import torch
torch.cuda.is_available()

True

GPUが使えるかチェックしましょう↑

## finetuneのためのデータのダウンロードとdataframe化

./dataにライブドアニュースコーパスをダウンロードしてgzipを展開します

In [1]:
from pathlib import Path
import pandas as pd

import urllib
import tarfile

url = "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"
download_dir = Path('./download/')
download_dir.mkdir(exist_ok=True)

local_filename, headers = urllib.request.urlretrieve(url, download_dir / "ldcc-20140209.tar.gz")
extract_dir = download_dir / "ldcc-20140209"
with tarfile.open(local_filename, "r:gz") as tar:
    tar.extractall(extract_dir)

展開したデータからpandasのDataFrameを作ります

ここらへんは展開したフォルダの構造を見ながらpythonでよしなにやっていきます。
ぐぐるとコード例もよく見つかります

In [2]:
df = pd.DataFrame(columns=["category", "url", "time", "title", "text"])
for txt_path in extract_dir.glob('text/*/*.txt'):
    file_name = txt_path.name
    category_name = txt_path.parent.name

    if file_name in ["CHANGES.txt", "README.txt", "LICENSE.txt"]:
        continue

    text_all = txt_path.read_text()
    text_lines = text_all.split("\n")
    url, time, title, *article = text_lines
    article = "\n".join(article)

    df.loc[file_name] = [category_name, url, time, title, article]

df.reset_index(inplace=True)
df.rename(columns={"index": "filename"}, inplace=True)


categories = df.category.unique()

In [3]:
df.head()

,filename,category,url,time,title,text
0,it-life-hack-6380347.txt,it-life-hack,http://news.livedoor.com/article/detail/6380347/,2012-03-18T15:00:00+0900,CPUもGPUも祭だワッショイ！GIGABYTE板祭でマザボ＆グラボのレビューアーを大募集,ソーシャルレビューコミュニティ「zigsow」（ジグソー）は、「GIGABYTE板祭 Ret...
1,it-life-hack-6394776.txt,it-life-hack,http://news.livedoor.com/article/detail/6394776/,2012-03-23T09:55:00+0900,新iPad時代の定番はコレだ！iPhoneとiPadを同時充電＋ステレオスピーカー内蔵スタン...,iPhoneとiPadは、どちらも使っている人も多いが、新iPadが登場したことで、この傾向...
2,it-life-hack-6628356.txt,it-life-hack,http://news.livedoor.com/article/detail/6628356/,2012-06-05T17:00:00+0900,ニコニコ動画の音楽ダウンロード始まる！NicoSoundにJASRACに加えJRCの管理楽曲が対応,日本最大級の動画サービス「niconico」のコンテンツ「ニコニコ動画：Zero」から提供が...
3,it-life-hack-6830890.txt,it-life-hack,http://news.livedoor.com/article/detail/6830890/,2012-08-07T10:00:00+0900,購入してよかったの最高はEOS DIGITAL ブランド総合研究所「デジタル家電イメージ調査...,株式会社ブランド総合研究所は、10のデジタル家電分野154ブランド（商品名及び企業名）を対象...
4,it-life-hack-6719823.txt,it-life-hack,http://news.livedoor.com/article/detail/6719823/,2012-07-03T13:00:00+0900,自分の顔で画面ロックを解除！ドコモ、GALAXY S IIにAndroid 4.0を提供,ドコモは2012年7月3日、サムスン製のスマートフォン「GALAXY S II SC-02C...


In [4]:
categories

array(['it-life-hack', 'movie-enter', 'livedoor-homme', 'smax',
       'topic-news', 'kaden-channel', 'sports-watch', 'dokujo-tsushin',
       'peachy'], dtype=object)

## 事前学習モデルの準備


事前学習済みデータをダウンロードします

google driveからのダウンロードはかなりpythonコード化しにくいので手作業してください

https://github.com/yoheikikuta/bert-japanese#pretrained-models

ここから以下をダウンロードします
[![Image from Gyazo](https://i.gyazo.com/2c1a9f74194a20801197ed686601cb82.png)](https://gyazo.com/2c1a9f74194a20801197ed686601cb82)

真ん中のbz2は事前学習のために使ったwikipediaのページのデータなのでfinetuneにはいらないはずです。

wikipediaのアーカイブは古いのは消えていくので再現性の担保のために保存していると思われます

google driveからダウンロードしたデータを任意のフォルダに置いて、model_dirの変数にフォルダのpathを書いてください

In [5]:
# model_dir = '' # ここにダウンロードしたフォルダを書いてください
model_dir = '/mnt/d/yoheikikuta-bert-japanese/'

base_ckpt = 'model.ckpt-1400000'  # 拡張子は含めない

### modelのconfigの用意

vocab_sizeを指定する必要があるが、これは、トークンをidに対応させるためのtableのサイズのことである

これはwiki-ja.vocabにtsvとして入っているので、この行数がvocab_sizeとなる

In [21]:
! cat {model_dir}wiki-ja.vocab | wc -l

32000


In [7]:
! cat {model_dir}wiki-ja.vocab | head -n 20

<unk>	0
<s>	0
</s>	0
[PAD]	0
[CLS]	0
[SEP]	0
[MASK]	0
、	-3.00936
。	-3.28261
▁	-3.52378
の	-3.65896
は	-4.00699
が	-4.361
・	-4.43092
)	-4.47007
(	-4.54356
年	-4.57164
に	-4.67272
を	-4.69082
で	-4.87051
cat: write error: Broken pipe


In [8]:
from transformers import BertConfig, BertForPreTraining, BertForSequenceClassification

vocab_size = 32000
bertconfig = BertConfig.from_pretrained('bert-base-uncased',
                                        num_labels=len(categories),
                                        output_attentions = False,
                                        output_hidden_states = False,
                                       )
bertconfig.vocab_size = vocab_size

### モデルの用意と読み込み

tfの重みを読み込んで、save_pretrainedでストレージに保存してから、BertForSequenceClassificationで読み込む

こうすることで、手軽にtfの重みを使ってtransformersのモデルでfinetuneができるという寸法です

当然、sequence classificationにはpretrainingのモデルよりもpooler layer(transformer stackの後ろの層)があって、そこが存在しなくて読み込めないのでwarnされます

In [9]:
# BERTモデルの"ガワ"の用意 (全パラメーターはランダムに初期化されている)
pretrained = BertForPreTraining(bertconfig)
# TensorFlowモデルの重み行列を読み込む (数分程度かかる場合がある)
pretrained.load_tf_weights(bertconfig, model_dir + base_ckpt)

tmp_dir = './tmp'
pretrained.save_pretrained(tmp_dir)
model = BertForSequenceClassification.from_pretrained(tmp_dir)

Some weights of the model checkpoint at ./tmp were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

## データセットの変換

livedoorコーパスのDataFrameをモデルに流すためのデータセットに変換します

huggingfaceが出している、datasetsというライブラリを使います。

https://huggingface.co/docs/datasets/index.html

transformersもdatasetsも同じhuggingfaceが出しているので、値を流すのが楽です

まずはcategoryのカラムを数値idにして、labelsというカラムにします。

In [10]:
df[['category', 'text']].head()

,category,text
0,it-life-hack,ソーシャルレビューコミュニティ「zigsow」（ジグソー）は、「GIGABYTE板祭 Ret...
1,it-life-hack,iPhoneとiPadは、どちらも使っている人も多いが、新iPadが登場したことで、この傾向...
2,it-life-hack,日本最大級の動画サービス「niconico」のコンテンツ「ニコニコ動画：Zero」から提供が...
3,it-life-hack,株式会社ブランド総合研究所は、10のデジタル家電分野154ブランド（商品名及び企業名）を対象...
4,it-life-hack,ドコモは2012年7月3日、サムスン製のスマートフォン「GALAXY S II SC-02C...


In [11]:
df['labels'] = df['category'].astype('category').cat.codes
df[['category','labels', 'text']].head()

,category,labels,text
0,it-life-hack,1,ソーシャルレビューコミュニティ「zigsow」（ジグソー）は、「GIGABYTE板祭 Ret...
1,it-life-hack,1,iPhoneとiPadは、どちらも使っている人も多いが、新iPadが登場したことで、この傾向...
2,it-life-hack,1,日本最大級の動画サービス「niconico」のコンテンツ「ニコニコ動画：Zero」から提供が...
3,it-life-hack,1,株式会社ブランド総合研究所は、10のデジタル家電分野154ブランド（商品名及び企業名）を対象...
4,it-life-hack,1,ドコモは2012年7月3日、サムスン製のスマートフォン「GALAXY S II SC-02C...


text,とlabelsのカラムからdatasetを作ります

In [12]:
from datasets import Dataset

ds = Dataset.from_pandas(df[['text', 'labels']])
dataset = ds.train_test_split()
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 5525
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 1842
    })
})

## tokenizerの用意
tokenizerは事前学習に用いたものと同じでなければいけません。

今回はyoheikikuta氏のモデルなので、Sentencepieceを使います。

transformersのモデルの中にもsentencepieceを使ったものがあるので、このtokenizerを流用します。  
transformersのドキュメントとソースを見ながら,Albertのtokenizerを利用することにしました。

https://huggingface.co/transformers/_modules/transformers/models/albert/tokenization_albert.html#AlbertTokenizer

Albert自体とは何も関係がありません。

先程google dirveからダウンロードしたものの中にwiki-ja.modelがあるので、このpathを渡します

In [13]:
BASE_SPM = 'wiki-ja.model'

In [14]:
from transformers import AlbertTokenizer
tokenizer = AlbertTokenizer(model_dir + BASE_SPM)

datasetにtokenizeをかけます。  
これはmap関数を使うと便利で、batch化もoptionで指定するとできます

In [15]:
max_length = bertconfig.max_position_embeddings

def tokenize(examples):
    input_ids = tokenizer(examples["text"], max_length=max_length, padding="max_length",truncation=True,).input_ids
    return {
        "input_ids": input_ids,
        "labels": examples['labels']
    }

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=['text'])
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'labels'])

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

## finetuneの実行

transformersが提供してるTranierクラスを使ってfinetuneを行います

動作確認のため、epochs数は少なめで、他の数値も適当です

In [16]:
from transformers import Trainer, TrainingArguments


# Trainerのパラメータの準備
training_args = TrainingArguments(
    output_dir='./results',          # 出力フォルダ
    num_train_epochs=2,              # エポック数
    per_device_train_batch_size=4,  # 訓練のバッチサイズ
    per_device_eval_batch_size=4,   # 評価のバッチサイズ
    warmup_steps=500,                # 学習率スケジューラのウォームアップステップ数
    weight_decay=0.01,               # 重み減衰の強さ
    logging_dir='./logs',            # ログ保存フォルダ
    eval_accumulation_steps=1,
)

## 指標の用意

trainした結果の精度などを評価する際に使う指標を選びます。

huggingface/datasetsライブラリの中にお手軽に指標を用意するload_metricがあるのでこれを使います。

compute_metrics 関数を定義してTrainerにわたすことでtrainer.evaluate()で、eval_datasetのデータから指標を計算してくれます。

In [17]:
import numpy as np
from datasets import load_metric

f1 = load_metric("f1")
acc = load_metric("accuracy")
precision = load_metric("precision")
recall = load_metric("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    params = dict(predictions=predictions, references=labels,)
    return { 
        **acc.compute(**params),
        **f1.compute(**params, average="weighted"),
        **precision.compute(**params, average="weighted"),
        **recall.compute(**params, average="weighted"),
       }

In [18]:
from transformers import Trainer
# Trainerの準備
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],     # 訓練データセット
    eval_dataset=tokenized_dataset['test'],        # 評価データセット
    compute_metrics=compute_metrics,
)

今回のデータ量と設定やPCの環境にもよりますが、私の手元だとfinetuneに20minくらいかかりました

In [19]:
trainer.train()

***** Running training *****
  Num examples = 5525
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 2764


Step,Training Loss
500,1.125600
1000,0.510600
1500,0.400000
2000,0.209500
2500,0.171900


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2500
Configuration saved in ./results/checkpoint-2500/config.json
Model weights saved in ./results/checkpoint-2500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2764, training_loss=0.4536512129213634, metrics={'train_runtime': 1199.4706, 'train_samples_per_second': 9.212, 'train_steps_per_second': 2.304, 'total_flos': 2907559890892800.0, 'train_loss': 0.4536512129213634, 'epoch': 2.0})

In [20]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1842
  Batch size = 4


{'eval_loss': 0.30360835790634155,
 'eval_accuracy': 0.9375678610206297,
 'eval_f1': 0.9371333992019317,
 'eval_precision': 0.9377411727515823,
 'eval_recall': 0.9375678610206297,
 'eval_runtime': 42.5026,
 'eval_samples_per_second': 43.338,
 'eval_steps_per_second': 10.846,
 'epoch': 2.0}

すくないepoch数でも精度が93%出ていることが確認できました

## 参考文献

https://github.com/miyamonz/bert-japanese-finetune-example

このnotebookの前進にあたる素振り。DatasetとかTrainerを使ってない文コードが長いが、sentencepieceの書き方とか、今回AlbertTokenizerで楽をした部分の中身はこっちのほうがわかりやすい

学習ループはpytorchで手書きしてるが、なにか抜けてる箇所があるかもしれない。
transformersに頼ったほうがその点安心なのでこのnotebookを作った

https://radiology-nlp.hatenablog.com/entry/2020/01/18/013039

tfの重みをtransformersで読み込んでるあたり。ただしこのnotebookで示した方法のほうがシンプルでおすすめです

https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part14.html

別モデルですが、transformersのTrainerクラスの使い方などを参考にしました